In [39]:
function sis_triang_sup(A::Matrix{Float64}, b::Vector{Float64})
    X = zeros(length(b))
    for i in length(b):-1:1
        x = (b[i] - sum(X .* A[i, :])) / A[i, i]
        X[i] = x
    end
    return X
end

sis_triang_sup (generic function with 1 method)

In [40]:
function sis_triang_inf(A::Matrix{Float64}, b::Vector{Float64})
    X = zeros(length(b))
    X[1] = b[1]/A[1,1]
    for i in 2:length(b)
        soma = 0
        el = A[i, 1:(i-1)] .* X[1:(i-1)]
        soma += sum(el)
        X[i] = (b[i] - soma)/A[i,i]
    end
    return X
end


sis_triang_inf (generic function with 1 method)

In [41]:
function pivotamento(c::Int64, l::Int64, M::Matrix{Float64})
    # c para coluna atual e l para linha atual

    vetor = abs.(M[l:end, c])
    max = argmax(vetor)
    max_index = max + l - 1
    M[max_index, :], M[l, :] = M[l, :], M[max_index, :]  
    
    return (max + l - 1)
end

pivotamento (generic function with 1 method)

In [42]:
function eliminacao_gauss(A::Matrix{Float64}, b::Vector{Float64})

    M = hcat(A, b)
    
    for i in 1:length(M[:, 1])
        pivotamento(i, i, M)
        pivot_line = M[i, :]
        for j in (i+1):(length(M[1, :])-1)
            m = M[j, i]/pivot_line[i]
            M[j, :] -= m * pivot_line
        end
    end

    return M[:, 1:(end-1)], M[:, end]
    
end

eliminacao_gauss (generic function with 1 method)

In [43]:
function resolver_sistema(A::Matrix{Float64}, b::Vector{Float64})
    X = eliminacao_gauss(A, b)
    return sis_triang_sup(X[1], X[2])
end

resolver_sistema (generic function with 1 method)

In [64]:
function permutacao(A::Matrix{Float64})

    n = length(A[1, :])
    M = hcat(Vector(1:n), A)
    N = zeros(n,n+1)

    for i in 1:n
        linha = findmax(M[:, i+1])[2]
        N[i, :] = M[linha, :]
        M[linha, :] .= -Inf
    end

    P = zeros(n,n)
    ordem = convert.(Int16, N[:, 1])
    for (i, j) in enumerate(ordem)
        P[i, j] = 1
    end

    return N[:,2:end], P
end

permutacao (generic function with 1 method)

In [65]:
using LinearAlgebra

function decompLU(A::Matrix{Float64})
    perm = permutacao(A)
    U = perm[1]
    n = length(A[1,:])
    L = zeros(n, n)
    L[diagind(A)] .= 1  
    P = perm[2]

    for i in 1:n
        pivot_line = U[i, :]
        for j in (i+1):n
            m = U[j, i]/pivot_line[i]
            L[j, i] = m
            U[j, :] -= m * pivot_line
        end
        #U = round.(U, digits = 12)
    end 
    return L, U, P
end


decompLU (generic function with 1 method)

In [45]:
b = rand(1:100, 6)
b = convert.(Float64, b)

6-element Vector{Float64}:
 27.0
 78.0
 99.0
 29.0
 78.0
  5.0

In [44]:
A = rand(1:100, (6,6))
A = convert.(Float64, A)

6×6 Matrix{Float64}:
 11.0  11.0  55.0  84.0  85.0  60.0
 92.0   1.0  31.0  10.0  85.0  62.0
 46.0  54.0  30.0  42.0  79.0  81.0
 12.0   3.0  68.0  39.0  13.0  83.0
 90.0  34.0  13.0  43.0  82.0  17.0
 18.0  52.0  93.0  65.0  35.0  74.0

In [76]:
X = decompLU(A);
X[3]

6×6 Matrix{Float64}:
 0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0

In [71]:
D = X[3]* A - X[1] * X[2]
mapreduce(x-> abs(x), +, D)

1.1102230246251565e-14

In [72]:
gauss = resolver_sistema(A, b)

6-element Vector{Float64}:
  0.7737827381561798
  0.1904358312681018
 -1.4722241298108623
  0.6415973785182231
 -0.32602069914291654
  1.1863917084591251

In [73]:
Pb = X[3] * b 
y = sis_triang_inf(X[1], Pb)
x = sis_triang_sup(X[2], y)

6-element Vector{Float64}:
  0.7737827381561798
  0.1904358312681018
 -1.4722241298108623
  0.6415973785182231
 -0.32602069914291654
  1.1863917084591251

In [75]:
sum(abs, gauss - x)

0.0